# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [22]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np
import torch
from datasets import load_dataset

device = "cpu"
if (torch.cuda.is_available()): device="cuda"
print(f"device: {device}")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased") #AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

dataset = load_dataset("agentlans/grammar-classification")
dataset["train"][0]

device: cuda


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'text': 'We have carried out a transaction cost analysis to help us understand the quality of execution on the execution venues where we execute/place orders.',
 'grammar': 1}

In [23]:
def tokenizing_map(dataset_entry):
    return tokenizer(dataset_entry["text"], padding="max_length", truncation=True, return_tensors="pt").to(device)

# temp small datasets to quickly iterate on config
small_validation_dataset = dataset["validation"].select(range(500)).rename_column("grammar","labels")
small_train_dataset = dataset["train"].select(range(500)).rename_column("grammar","labels")

small_validation_dataset=small_validation_dataset.map(tokenizing_map, batched=True)
small_train_dataset=small_train_dataset.map(tokenizing_map, batched=True)

for param in model.base_model.parameters():
    param.requires_grad = False
    
small_train_dataset

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 500
})

In [24]:
from datasets import load_metric

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
#     accuracy_metric = load_metric("accuracy")
#     f1_metric = load_metric("f1")
#     precision_metric = load_metric("precision")
#     recall_metric = load_metric("recall")
#     accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
#     f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')  # Use 'macro', 'micro', or 'weighted'
#     precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
#     recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')

    return {
        "accuracy": (predictions == labels).mean(),
#         "a1": accuracy["accuracy"],
#         "f1": f1["f1"],
#         "precision": precision["precision"],
#         "recall": recall["recall"]
    }

trainer = Trainer(
    model=model,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    args=TrainingArguments(
        output_dir="./data/grammar",
        evaluation_strategy="epoch",
        num_train_epochs=3
    ),
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    compute_metrics=compute_metrics
)
# trainer.train()
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6919270157814026,
 'eval_accuracy': 0.53,
 'eval_runtime': 7.8782,
 'eval_samples_per_second': 63.466,
 'eval_steps_per_second': 7.997}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [26]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    task_type="CAUSAL_LM",
    target_modules=["q_lin","k_lin","v_lin"]
)

lora_model = get_peft_model(model, config)
lora_model.to(device)
print("done")

done


In [29]:
from transformers import default_data_collator
lora_trainer = Trainer(
    model=lora_model,
    data_collator=default_data_collator,
    args=TrainingArguments(
        output_dir="./data/grammar",
        evaluation_strategy="epoch",
        num_train_epochs=5,
        learning_rate=2e-6,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
    ),
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    compute_metrics=compute_metrics
)
lora_model.print_trainable_parameters()
# print ( lora_trainer.train_dataset[0])
# print((small_train_dataset[0]))
# print((small_validation_dataset[0]))
lora_trainer.train()
lora_trainer.evaluate()

trainable params: 221,184 || all params: 67,176,194 || trainable%: 0.3292594992803552


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.692259,0.540000
2,No log,0.692268,0.544000
3,No log,0.692273,0.546000


KeyboardInterrupt: 

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [ ]:
# Saving the model
44model.save("/tmp/your_model_name")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.